# 第6章 アイテムベース協調フィルタリング

# 準備

In [1]:
import pprint
import numpy as np
np.set_printoptions(precision=3)

# 近傍アイテム数
K_ITEMS = 3
# 閾値
THETA = 0

R = np.array([
              [np.nan, 4,      3,      1,      2,      np.nan],
              [5,      5,      4,      np.nan, 3,      3     ],
              [4,      np.nan, 5,      3,      2,      np.nan],
              [np.nan, 3,      np.nan, 2,      1,      1     ],
              [2,      1,      2,      4,      np.nan, 3     ],
])
U = np.arange(R.shape[0])
I = np.arange(R.shape[1])
Ui = [U[~np.isnan(R)[:,i]] for i in I]
Iu = [I[~np.isnan(R)[u,:]] for u in U]
ru_mean = np.nanmean(R, axis=1)
R2 = R - ru_mean.reshape((ru_mean.size, 1))

# コサイン類似度

## 01 アイテムiとアイテムjのコサイン類似度

In [2]:
def cos(i, j):
    """
    評価値行列Rにおけるアイテムiとアイテムjのコサイン類似度を返す。

    Parameters
    ----------
    i : int
        アイテムiのID
    j : int
        アイテムjのID

    Returns
    -------
    float
        コサイン類似度
    """
    Uij = np.intersect1d(Ui[i], Ui[j])
    
    cosine = np.sum([R[u, i] * R[u, j] for u in Uij]) / \
        (np.sqrt(np.sum([R[u, i] **2 for u in Uij])) * np.sqrt(np.sum([R[u, j] **2 for u in Uij])))
    return cosine

In [3]:
i = 0
j = 4
cosine = cos(i, j)
print('cos({}, {}) = {:.3f}'.format(i, j, cosine))

cos(0, 4) = 0.996


# 調整コサイン類似度

## 02 アイテムiとアイテムjの調整コサイン類似度

In [4]:
def adjusted_cos(i, j):
    """
    評価値行列R2におけるアイテムiとアイテムjの調整コサイン類似度を返す。

    Parameters
    ----------
    i : int
        アイテムiのID
    j : int
        アイテムjのID

    Returns
    -------
    cosine : float
        調整コサイン類似度
    """
    Uij = np.intersect1d(Ui[i], Ui[j])
    
    cosine = np.sum([R2[u, i] * R2[u, j] for u in Uij]) / \
        (np.sqrt(np.sum([R2[u, i] **2 for u in Uij])) * np.sqrt(np.sum([R2[u, j] **2 for u in Uij])))
    return cosine

In [5]:
i = 0
j = 4
cosine = adjusted_cos(i, j)
print('cos({}, {})\' = {:.3f}'.format(i, j, cosine))

cos(0, 4)' = -0.868


# アイテム-アイテム類似度行列

In [6]:
def sim(i, j):
    """
    アイテム類似度関数：アイテムiとアイテムjのアイテム類似度を返す。

    Parameters
    ----------
    i : int
        アイテムiのID
    j : int
        アイテムjのID

    Returns
    -------
    float
        アイテム類似度
    """
    return adjusted_cos(i, j)

## 03 アイテム-アイテム類似度行列

In [7]:
S = np.zeros((I.size, I.size))
for i in I:
    for j in I:
        S[i, j] = sim(i, j)
print('S = \n{}'.format(S))

S = 
[[ 1.     0.842  0.494 -0.829 -0.868 -0.987]
 [ 0.842  1.     0.896 -0.788 -0.91  -0.942]
 [ 0.494  0.896  1.    -0.583 -0.845 -0.514]
 [-0.829 -0.788 -0.583  1.     0.469  0.497]
 [-0.868 -0.91  -0.845  0.469  1.     1.   ]
 [-0.987 -0.942 -0.514  0.497  1.     1.   ]]


# 類似アイテムの選定

## 04 類似度上位k件のアイテム集合

## 05 類似度がしきい値以上のアイテム集合

In [8]:
# アイテム-アイテム類似度行列から対象アイテムを除外した辞書
Ii = {i: {j: S[i,j] for j in I if i != j} for i in I}
print('Ii = ')
pprint.pprint(Ii)
Ii = {i: dict(sorted(Ii[i].items(), key=lambda x: x[1], reverse=True)[:K_ITEMS]) for i in I}
print('Ii = ')
pprint.pprint(Ii)
Ii = {i: {k: v for k, v in Ii[i].items() if v >= THETA} for i in I}
print('Ii = ')
pprint.pprint(Ii)
# 各アイテムの類似アイテム集合をまとめた辞書
Ii = {i: np.array(list(Ii[i].keys())) for i in I}
print('Ii = ')
pprint.pprint(Ii)

Ii = 
{0: {1: 0.8418791389638738,
     2: 0.49365474375598073,
     3: -0.8291725540450335,
     4: -0.8682431421244593,
     5: -0.987241120712647},
 1: {0: 0.8418791389638738,
     2: 0.896314672184623,
     3: -0.7876958617794716,
     4: -0.9099637547345425,
     5: -0.9419581446623225},
 2: {0: 0.49365474375598073,
     1: 0.896314672184623,
     3: -0.5833076828172804,
     4: -0.8451542547285166,
     5: -0.5144957554275266},
 3: {0: -0.8291725540450335,
     1: -0.7876958617794716,
     2: -0.5833076828172804,
     4: 0.4685212856658182,
     5: 0.49665813370370504},
 4: {0: -0.8682431421244593,
     1: -0.9099637547345425,
     2: -0.8451542547285166,
     3: 0.4685212856658182,
     5: 1.0},
 5: {0: -0.987241120712647,
     1: -0.9419581446623225,
     2: -0.5144957554275266,
     3: 0.49665813370370504,
     4: 1.0}}
Ii = 
{0: {1: 0.8418791389638738, 2: 0.49365474375598073, 3: -0.8291725540450335},
 1: {0: 0.8418791389638738, 2: 0.896314672184623, 3: -0.7876958617794716},
 2

# 嗜好予測

## 06 類似アイテム集合の中でユーザuが評価値を与えているアイテム集合

## 07 予測評価値

In [9]:
def predict(u, i):
    """
    予測関数：ユーザuのアイテムiに対する予測評価値を返す。

    Parameters
    ----------
    u : int
        ユーザuのID
    i : int
        アイテムiのID
    
    Returns
    -------
    float
        ユーザuのアイテムiに対する予測評価値
    """
    Iiu = np.intersect1d(Ii[i], Iu[u])
    print('I{}{} = {}'.format(i, u, Iiu))

    if Iiu.size <= 0: return ru_mean[u]
    rui_pred = np.sum(S[i, Iiu] * R[u, Iiu]) / np.sum(np.abs(S[i, Iiu]))
    
    return rui_pred

In [10]:
u = 0
i = 0
print('r{}{} = {:.3f}'.format(u, i, predict(u, i)))
u = 0
i = 5
print('r{}{} = {:.3f}'.format(u, i, predict(u, i)))

I00 = [1 2]
r00 = 3.630
I50 = [3 4]
r05 = 1.668
